In [1]:
import pandas_datareader as pdr
import datetime
import numpy as np 
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import time
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
import pytz
from yahooquery import Ticker
from matplotlib import pyplot
import matplotlib.cm as cm

/home/thomas/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
from ibapi import wrapper
from ibapi.client import EClient
from ibapi.utils import iswrapper #just for decorator
from ibapi.common import *

from ib.ext.Contract import Contract
from ib.ext.Order import Order
from ib.ext import EWrapper
from ib.opt import Connection, message
import ib

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

# aux functions

In [31]:
def buy_order(data, current_max, wanted_drop, purchase_price):
            
    buy_mask = 0
    current_price = data[-1]
                
    if current_price <= (1-wanted_drop/100)*current_max and current_price <= purchase_price:
        buy_mask = 1
    else:
        buy_mask = 0
                    
    return buy_mask, current_price

In [32]:
def sell_order(data, purchase_price, wanted_increase):
    
    current_price = data[-1]
    sell_mask = 0
    
    if current_price >= (1+(wanted_increase/2)/100)*purchase_price:
        sell_mask = 1
                
    return sell_mask

# fetch company names and price

In [6]:
# new strategy of looking mostly at the market opening

def vantage_fetch_latest_value(company_symbols_list):

    list_data = []
    for company in company_symbols_list:
        try:
            time.sleep(15)
            print(company)
            ts = TimeSeries(key='XQJSCNK7QEMWH6RZ', output_format='pandas')
            data, meta_data = ts.get_intraday(symbol=company, interval='1min', outputsize='full')
            data = np.flip(data["4. close"])
            
            list_of_days = np.unique(data.index.strftime("%D"))
            for day in list_of_days:
                subset_data = data[data.index.strftime("%D") == day].values
                list_data.append(subset_data)
            
        except ValueError:
            print("company symbol " + company + " not a valid API call")

    return list_data, meta_data

In [7]:
# using yahoo query API

def yahooquery_fetch_latest_value(company_symbols_list):
    
    for company in company_symbols_list:
        try:
            tickers = Ticker(company)
            dic_data = tickers.financial_data
            data = dic_data[company]["currentPrice"]

        except ValueError:
            print("company symbol " + company + " not a valid API call")

    return data

In [24]:
# mainstream stocks
company_symbols_list = ["NCLH", "CCL"]

meta_list_data = []
for company in company_symbols_list:
    list_data, _ = vantage_fetch_latest_value([company])
    meta_list_data.append(list_data)

15-Jun-20 15:02:34 DEBUG     Starting new HTTPS connection (1): www.alphavantage.co:443


NCLH


15-Jun-20 15:02:35 DEBUG     https://www.alphavantage.co:443 "GET /query?function=TIME_SERIES_INTRADAY&symbol=NCLH&interval=1min&outputsize=full&apikey=XQJSCNK7QEMWH6RZ&datatype=json HTTP/1.1" 200 None
15-Jun-20 15:02:56 DEBUG     Starting new HTTPS connection (1): www.alphavantage.co:443


CCL


15-Jun-20 15:02:57 DEBUG     https://www.alphavantage.co:443 "GET /query?function=TIME_SERIES_INTRADAY&symbol=CCL&interval=1min&outputsize=full&apikey=XQJSCNK7QEMWH6RZ&datatype=json HTTP/1.1" 200 None


# analysis of the parameters: drop_wanted and increase_wanted

In [27]:
for company_data in meta_list_data:
    count = 0
    for data in company_data:
        change = data[-1]-data[0]
        if change > 0:
            count += 1
    print(str(count) + "/" + str(len(company_data)))

3/5
3/5


In [28]:
# absolute drop wanted

percent_wanted_list = []
max_price_list = []

for company_data in meta_list_data:
    company_data = company_data[-1]
    max_list = []
    std_list = []
    mean_list = []
    for data in [company_data]:
        std_list.append(np.std(data))
        mean_list.append(np.mean(data))
        max_list.append(np.max(data))

    percent_wanted = np.mean(std_list)*100/np.mean(mean_list)
    weekly_max = np.max(max_list)
    
    percent_wanted_list.append(percent_wanted)
    max_price_list.append(weekly_max)

print(percent_wanted_list, max_price_list)

[2.32620115971922, 1.9918608403648226] [20.48, 19.798]


In [191]:
# # percentage increase wanted

# count = 0
# for list_data in meta_list_data:
#     print("Company: " + company_symbols_list[count])
#     percent_wanted = percent_wanted_list[count]
#     for data in list_data:
    
#         list_indices = []
#         list_indices.append(0)
#         occurence_index = 0
#         current_purchase_price = data[0]

#         for data_point in data:
#             if data_point <= current_purchase_price*(1-percent_wanted/100) or data_point >= current_purchase_price*(1+percent_wanted/100):
#                 current_purchase_price = data_point
#                 list_indices.append(occurence_index)
#             occurence_index += 1

#         plt.plot()
#         plt.plot(data)
#         for index in range(len(list_indices)):
#             plt.scatter(list_indices[index], data[list_indices[index]], marker = "+", s = 200)
#         plt.show() 
#         print(len(list_indices)-1)
        
#     count += 1


# test the strategy

In [209]:
count = 0

for list_data in meta_list_data:
    company = company_symbols_list[count]
    print("for company: " + company)
    
    for data in list_data:

        daily_purchase_price = 10000
        current_max = max_price_list[count]
        percent_wanted = percent_wanted_list[count]
        
        buy_price_list = []
        sell_price_list = []

        time_series = []
        purchase = False

        counter = 0
        for data_point in data:
            sell_mask = 0
            buy_mask = 0

            time_series.append(data_point)

            if counter >= 30:
                if purchase == False:

                    if data_point > current_max:
                        current_max = data_point

                    buy_mask, purchase_price = buy_order(time_series, current_max, percent_wanted, daily_purchase_price)
                    if buy_mask:
                        purchase = True  
                        daily_purchase_price = purchase_price
                else:
                    sell_mask = sell_order(time_series, purchase_price, percent_wanted)
                    if sell_mask:
                        purchase = False

            if buy_mask:
                buy_price_list.append(data_point)

            if sell_mask:
                sell_price_list.append(data_point)

            counter += 1

        print(buy_price_list, sell_price_list)
        
    count += 1


for company: AAPL
[320.3604, 320.25] [321.451, 321.26]
[321.51] [322.71]
[324.43] [325.5154]
[324.7001] []
[325.985] [327.25]
for company: BABA
[205.9] [206.71]
[210.19] [211.06]
[216.19, 216.13] [217.12, 217.0]
[218.62, 218.61, 218.6] [219.58, 219.475]
[219.35] []
for company: BAC
[24.52, 24.52] [24.69]
[25.2779] []
[25.685] [25.85]
[26.235, 26.225] [26.42, 26.4]
[28.035, 28.02, 28.015] [28.2223, 28.225, 28.2]
for company: AMD
[53.75] []
[53.168] [53.55]
[53.0] []
[53.35, 53.29] [53.67]
[52.4484] [52.74]
for company: DAL
[26.235, 26.1684] [26.645]
[26.47] []
[27.451] [27.875]
[30.255] [30.8084]
[35.47, 35.27, 35.2243] [36.067, 35.8124]


# real live trading

In [11]:
def get_time():

    tz_NY = pytz.timezone('America/New_York') 
    datetime_NY = datetime.now(tz_NY)
    formatted_time = datetime_NY.strftime("%H:%M:%S")
    str_date = datetime.strptime(formatted_time, "%H:%M:%S")
    
    return str_date

In [12]:
def error_handler(msg):
    """Handles the capturing of error messages"""
    print("Server Error: %s" % msg)

def reply_handler(msg):
    """Handles of server replies"""
    print("Server Response: %s, %s" % (msg.typeName, msg))

In [13]:
# make the contract

def create_contract(symbol, sec_type, exch, prim_exch, curr):
    
    contract = Contract()
    contract.m_symbol = symbol
    contract.m_secType = sec_type
    contract.m_exchange = exch
    contract.m_primaryExch = prim_exch
    contract.m_currency = curr
    
    return contract

In [14]:
# define 3 types of orders, to make the overall buy limit, sell limit (profit taker) and trailing stop

def make_market_order(action, quantity, order_id, transmit, tif):
    
    order = Order()
    order.m_orderType = 'MKT'
    order.m_totalQuantity = quantity
    order.m_action = action
    order.m_orderId = order_id
    order.m_transmit = transmit
    order.m_tif = tif
    
    return order

def make_limit_order(action, quantity, price, order_id, transmit, tif = None, parent_id = None):

    order = Order()
    order.m_orderType = 'LMT'
    order.m_totalQuantity = quantity
    order.m_action = action
    order.m_lmtPrice = price
    order.m_orderId = order_id
    order.m_transmit = transmit
    
    if tif != None:
        order.m_tif = tif
    
    if parent_id != None:
        order.m_parentId = parent_id

    return order

def make_stop_order(action, quantity, order_id, transmit, trail_percentage, parent_id):

    order = Order()
    order.m_orderType = 'TRAIL'
    order.m_totalQuantity = quantity
    order.m_action = action
    order.m_orderId = order_id
    order.m_transmit = transmit
    order.m_trailingPercent = trail_percentage
    order.m_parentId = parent_id
    
    return order

In [15]:
########################################

def decision(time_series, purchase, current_max, percent_wanted, daily_purchase_price):

    buy_mask = 0
    sell_mask = 0
    
    if purchase == False:

        buy_mask, purchase_price = buy_order(time_series, current_max, percent_wanted, daily_purchase_price)
            
        if buy_mask:
            purchase = True  
            daily_purchase_price = purchase_price
    else:
        sell_mask = sell_order(time_series, daily_purchase_price, percent_wanted)
        if sell_mask:
            purchase = False
    
    return buy_mask, sell_mask, purchase, daily_purchase_price
        
################################

In [21]:
order_id = 1

In [33]:
amount_to_trade = 140
company_symbols_list = ["NCLH"]

In [23]:
if __name__ == "__main__":
    
    time_series_list = [[] for _ in range(len(company_symbols_list))]
    buy_mask_list = [[] for _ in range(len(company_symbols_list))]
    sell_mask_list = [[] for _ in range(len(company_symbols_list))]
    purchase_bool_list = [False]*len(company_symbols_list)
    lag_counter_list = [0]*len(company_symbols_list)
    
    daily_purchase_price = [10000]*len(company_symbols_list)
    max_price_list = max_price_list
    percent_wanted_list = percent_wanted_list

    current_time = get_time()

    while current_time > datetime(1900, 1, 1, 10, 0, 0) and current_time < datetime(1900, 1, 1, 16, 0, 0):

        t1 = time.time()

        # make the buy or sell decision at current time t
        company_counter = 0
        for company in company_symbols_list:
            print(company)
            latest_price = yahooquery_fetch_latest_value([company])
            time_series_list[company_counter].append(latest_price)
            
            if latest_price > max_price_list[company_counter]:
                max_price_list[company_counter] = latest_price
                    
            # morning and lunch strategies
            buy_mask, sell_mask, purchase, purchase_price = decision(time_series_list[company_counter], 
                                     purchase_bool_list[company_counter], max_price_list[company_counter], 
                                     percent_wanted_list[company_counter], daily_purchase_price[company_counter])  
            
            buy_mask_list[company_counter] = buy_mask
            sell_mask_list[company_counter] = sell_mask
            purchase_bool_list[company_counter] = purchase
            daily_purchase_price[company_counter] = purchase_price

            company_counter += 1
    
        purchase_list_indices = []
        for index in range(len(company_symbols_list)):
            if buy_mask_list[index] == 1:
                purchase_list_indices.append(index)

        if purchase_list_indices != []:
            
            for index in purchase_list_indices:
                company = company_symbols_list[index]
                percent_wanted = percent_wanted_list[index]
                    
                latest_price = time_series_list[index][-1]
                
                tws_conn = Connection.create(<ip>, port=7496, clientId=25237812)
                tws_conn.connect()
                tws_conn.register(error_handler, 'Error')
                tws_conn.registerAll(reply_handler)

                quantity = int(amount_to_trade/latest_price)
                order_id = order_id

                my_contract = create_contract(company, 'STK', 'SMART', 'SMART', 'USD')

                # limit buy order
                my_order_buy = make_limit_order("BUY", quantity, np.round(latest_price+0.02, 2), order_id, False, "DAY")
                tws_conn.placeOrder(order_id, my_contract, my_order_buy)
                
                # associated limit sell order
                my_order_sell = make_limit_order("SELL", quantity, np.round(latest_price*(1+percent_wanted/200)-0.02, 2), order_id+1, True, None, order_id)
                tws_conn.placeOrder(order_id+1, my_contract, my_order_sell)

                time.sleep(2)
                order_id += 2

                tws_conn.disconnect()
        
        current_time = get_time()  
        t2 = time.time()
        tick_time = t2 - t1
        print("compute time was: " + str(tick_time))

        if tick_time > 59.9:
            pass
        else:
            time.sleep(59.9-tick_time)

15-Jun-20 13:35:58 DEBUG     Starting new HTTPS connection (1): query2.finance.yahoo.com:443


NCLH


15-Jun-20 13:35:58 DEBUG     https://query2.finance.yahoo.com:443 "GET /v10/finance/quoteSummary/NCLH?modules=financialData&formatted=false&lang=en-US&region=US&corsDomain=finance.yahoo.com HTTP/1.1" 200 450


Server Version: 76
TWS Time at connection:20200615 13:35:58 EST
Server Response: managedAccounts, <managedAccounts accountsList=U3975864>
Server Response: nextValidId, <nextValidId orderId=1>
Server Error: <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfarm>
Server Response: error, <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfarm>
Server Error: <error id=-1, errorCode=2106, errorMsg=HMDS data farm connection is OK:ushmds>
Server Response: error, <error id=-1, errorCode=2106, errorMsg=HMDS data farm connection is OK:ushmds>
Server Error: <error id=-1, errorCode=2158, errorMsg=Sec-def data farm connection is OK:secdefil>
Server Response: error, <error id=-1, errorCode=2158, errorMsg=Sec-def data farm connection is OK:secdefil>
Server Response: openOrder, <openOrder orderId=2, contract=<ib.ext.Contract.Contract object at 0x7f4c13494e10>, order=<ib.ext.Order.Order object at 0x7f4c15049a58>, orderState=<ib.ext.OrderState.OrderState 

KeyboardInterrupt: 

In [36]:
int(amount_to_trade/25)

5